In [1]:
import pandas as pd
import new_lib as nl
import os
import env

In [11]:
def acquire_zillow():
    if os.path.isfile('zillow.csv'):
        zil = pd.read_csv('zillow.csv', index_col= 0)
    else:
        zil = nl.connect('zillow', 'zillow.csv', '''SELECT * FROM properties_2017 as pr
JOIN predictions_2017 AS pe ON pr.parcelid = pe.parcelid
JOIN propertylandusetype AS l ON pr.propertylandusetypeid = l.propertylandusetypeid
WHERE pe.transactiondate >= 01-01-2017 AND l.propertylandusetypeid = 261;''')
    return zil

In [13]:
def prep_zillow(zil):
    zil = zil.rename(columns = {'bedroomcnt': 'bedrooms', 'bathroomcnt': 'bathrooms', 
                            'calculatedfinishedsquarefeet': 'square_footage', 
                            'taxvaluedollarcnt': 'tax_value', 'taxamount': 'tax'})
    zil = zil[['id', 'bedrooms', 'bathrooms', 'square_footage', 'tax_value', 'yearbuilt', 
           'tax', 'fips']]
    # getting dataframe into the right subset
    zil = zil.dropna()
    zil = zil.reset_index()
    zil = zil.drop(columns = 'index')
    zil.yearbuilt = zil.yearbuilt.astype(int)
    zil.index.name = 'index'
    return zil

In [16]:
def wrangle_zillow():
    zil = prep_zillow(acquire_zillow())
    zil = nl.remove_outliers(zil)
    return zil